# Twitter

### Pre-processing

TODO
- Clean duplicates

In [36]:
import pandas as pd
import os
import re

filelist = os.listdir("data")
matching = [s for s in filelist if "dataset" in s]
matching.sort(reverse=True)
print(matching[0])

df = pd.read_csv("data/"+matching[0])
display(df)

dataset_17:19:41_22-12-2022.csv


,Unnamed: 0,X,y
0,0,Claudia makes dishonesty her trademark. https:...,1
1,1,@autumnbueno @alexbruesewitz @PapiTrumpo THAT'...,1
2,2,I used to have a nice amount in my IRA after 4...,1
3,3,@CryptoNinjaco @Starinne @DC_Draino @KariLake ...,1
4,4,https://t.co/ITCjqOVvle EXCLUSIVE: National ...,1
...,...,...,...
2995,995,Have a scary right and get connect your really...,2
2996,996,A good name is earlier y p x #推特买粉 IE R I Ll...,2
2997,997,have a good sleep mark&lt;3 https://t.co/ySxW9...,2
2998,998,RT @anumeet_insan: @ThathSingh777 @Gurmeetramr...,2


In [67]:
df.drop_duplicates(subset='X', keep='first', inplace=False)

,Unnamed: 0,X,y
0,0,Claudia makes dishonesty her trademark. https:...,1
1,1,@autumnbueno @alexbruesewitz @PapiTrumpo THAT'...,1
2,2,I used to have a nice amount in my IRA after 4...,1
3,3,@CryptoNinjaco @Starinne @DC_Draino @KariLake ...,1
4,4,https://t.co/ITCjqOVvle EXCLUSIVE: National ...,1
...,...,...,...
2995,995,Have a scary right and get connect your really...,2
2996,996,A good name is earlier y p x #推特买粉 IE R I Ll...,2
2997,997,have a good sleep mark&lt;3 https://t.co/ySxW9...,2
2998,998,RT @anumeet_insan: @ThathSingh777 @Gurmeetramr...,2
